# Greenblatt Magic formula

## The Magic Formula: Explanation and Formula

The **Magic Formula** is an investment strategy developed by **Joel Greenblatt** to identify high-quality companies that are also undervalued. It ranks companies based on two key financial ratios:

---

## 📌 Core Idea

> **Buy good companies at cheap prices.**

The strategy identifies:
- **"Good companies"** → those with high returns on capital (efficient use of capital).
- **"Cheap companies"** → those with high earnings yield (undervalued based on operating profits).

---

## 🧮 The Formula

1. **Earnings Yield (EY):**
$$
\text{Earnings Yield} = \frac{\text{EBIT}}{\text{Enterprise Value}}
$$
- Measures how cheap the stock is.
- EBIT = Earnings Before Interest and Taxes
- Enterprise Value = Market Cap + Debt - Cash

2. **Return on Capital (ROC):**
$$
\text{Return on Capital} = \frac{\text{EBIT}}{\text{Net Working Capital} + \text{Net Fixed Assets}}
$$
- Measures the quality of the business (how efficiently it uses its capital).

---

## 🔍 Implementation Steps

1. **Filter the universe**: Remove financials, utilities, and companies with very small market cap.
2. **Rank all remaining stocks** by:
   - Earnings Yield (high = better)
   - Return on Capital (high = better)
3. **Compute combined rank**:
   $$
   \text{Combined Rank} = \text{Rank}_{EY} + \text{Rank}_{ROC}
   $$
4. **Sort by Combined Rank** (lowest = best overall).
5. **Pick top N stocks** (e.g., top 20–30).
6. **Hold for 1 year**, rebalance annually.

---

## ✅ Why It Works

- Avoids paying too much for popular stocks.
- Focuses on operationally efficient, consistently profitable companies.
- Enforces a disciplined, rules-based approach.

---

## ⚠️ Notes and Caveats

- Avoids subjective judgement; however, **screening accuracy** depends on **quality of financial data**.
- May underperform in short-term or irrational markets.
- Works best over a multi-year horizon (3–5+ years).
- I have added a minimum 40% margin restriction to the original formula. 

---


In [1]:
import os
from tenacity import retry, stop_after_attempt, wait_exponential
import requests
import pandas as pd
from tqdm import tqdm

api_key = os.getenv('financial_modeling_prep_api_key')
assert api_key is not None

In [10]:


# Retry settings: 5 attempts with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=10, max=60))
def fetch_json(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_magic_formula(api_key, symbols, minimum_margin=0.4, minimum_marketcap=500_000_000, minimum_revenue=100_000_000):
    import pandas as pd
    from tqdm import tqdm

    base_url = "https://financialmodelingprep.com/stable"
    results = []

    for symbol in tqdm(symbols, desc="Processing symbols"):
        try:
            # Step 1: Check market cap
            profile_url = f"{base_url}/enterprise-values?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            if not profile_data or profile_data[0].get('marketCapitalization') < minimum_marketcap:
                continue

            # Step 2: Exclude financials and real estate sectors
            profile_url = f"{base_url}/search-exchange-variants?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            if not profile_data or profile_data[0].get('sector') in ['Financial Services', 'Real Estate']:
                continue

            # Step 3: Fetch income statement
            income_url = f"{base_url}/income-statement/?symbol={symbol}&apikey={api_key}"
            income_data = fetch_json(income_url)[0]

            # Step 4: Fetch balance sheet
            balance_url = f"{base_url}/balance-sheet-statement/?symbol={symbol}&apikey={api_key}"
            balance_data = fetch_json(balance_url)[0]

            # Step 5: Fetch enterprise value
            ev_url = f"{base_url}/enterprise-values/?symbol={symbol}&apikey={api_key}"
            ev_data = fetch_json(ev_url)[0]

            # Extract relevant fields
            ebit = income_data.get('ebit')
            revenue = income_data.get('revenue')
            total_assets = balance_data.get('totalAssets')
            current_liabilities = balance_data.get('totalCurrentLiabilities')
            enterprise_value = ev_data.get('enterpriseValue')
            net_income = income_data.get('netIncome')
            operating_expenses = income_data.get('operatingExpenses')
            other_expenses = income_data.get('otherExpenses')

            # Skip if net income is non-positive
            if net_income is None or net_income <= 0:
                continue

            # Ensure all required fields are present
            if None in (ebit, revenue, total_assets, current_liabilities, enterprise_value):
                print(f"missing data for {symbol}")
                continue

            # Filter: minimum revenue
            if revenue < minimum_revenue:
                continue

            # Filter: operating expenses must be non-negative
            if operating_expenses is not None and operating_expenses < 0:
                continue

            # Filter: exclude extreme accounting adjustments
            if other_expenses is not None and other_expenses < -revenue:
                continue

            # Filter: remove absurdly high EBIT margins (e.g., > 200%)
            ebit_margin = ebit / revenue
            if ebit_margin < minimum_margin or ebit_margin > 2:
                continue

            # Compute Greenblatt metrics
            earnings_yield = ebit / enterprise_value
            capital = total_assets - current_liabilities
            if capital <= 0:
                continue
            return_on_capital = ebit / capital

            # Append result
            results.append({
                'symbol': symbol,
                'ebit_margin': ebit_margin,
                'earnings_yield': earnings_yield,
                'return_on_capital': return_on_capital
            })

        except Exception as e:
            print(f'Symbol: {symbol}\n')
            print(e)

    # Step 6: Compile DataFrame and rank
    df = pd.DataFrame(results)
    if df.empty:
        return df

    df['ey_rank'] = df['earnings_yield'].rank(ascending=False)
    df['roc_rank'] = df['return_on_capital'].rank(ascending=False)
    df['combined_rank'] = df['ey_rank'] + df['roc_rank']
    df_sorted = df.sort_values(by='combined_rank')

    return df_sorted[['symbol', 'ebit_margin', 'earnings_yield', 'return_on_capital', 'combined_rank']]


In [11]:

symbols = pd.read_csv("russel1000.csv")
symbols = symbols['Ticker'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.15)
print(top_stocks)
russel1000_filtered = top_stocks

Processing symbols:   1%|▋                                                            | 11/1012 [00:05<07:42,  2.16it/s]

Symbol: BRKB

'<' not supported between instances of 'NoneType' and 'int'


Processing symbols:  82%|█████████████████████████████████████████████████▎          | 832/1012 [09:35<00:44,  4.05it/s]

Symbol: BFB

'<' not supported between instances of 'NoneType' and 'int'


Processing symbols:  97%|██████████████████████████████████████████████████████████▏ | 982/1012 [11:21<00:10,  2.90it/s]

Symbol: BFA

'<' not supported between instances of 'NoneType' and 'int'


Processing symbols:  98%|██████████████████████████████████████████████████████████▋ | 989/1012 [11:24<00:09,  2.40it/s]

Symbol: LENB

'<' not supported between instances of 'NoneType' and 'int'


Processing symbols: 100%|███████████████████████████████████████████████████████████| 1012/1012 [11:30<00:00,  1.47it/s]

    symbol  ebit_margin  earnings_yield  return_on_capital  combined_rank
56      MO     0.723048        0.132416           0.560009           16.0
212   BBWI     0.183386        0.106695           0.368031           45.0
216    HRB     0.233053        0.102539           0.375378           48.0
236  LBRDA     1.255906        0.089240           0.379649           53.0
204  LBRDK     1.255906        0.089240           0.379649           53.0
..     ...          ...             ...                ...            ...
120    FIS     0.162733        0.029762           0.059516          422.0
92     SRE     0.239590        0.035089           0.036529          424.0
232   CWEN     0.199854        0.026133           0.020131          453.0
226     AL     0.215126        0.023425           0.019051          458.0
51     ADI     0.222715        0.017863           0.046410          461.0

[238 rows x 5 columns]


In [12]:
russel1000_filtered.head(10)

,symbol,ebit_margin,earnings_yield,return_on_capital,combined_rank
56,MO,0.723048,0.132416,0.560009,16.0
212,BBWI,0.183386,0.106695,0.368031,45.0
216,HRB,0.233053,0.102539,0.375378,48.0
236,LBRDA,1.255906,0.089240,0.379649,53.0
204,LBRDK,1.255906,0.089240,0.379649,53.0
157,PHM,0.223233,0.170498,0.252484,54.0
108,FAST,0.200822,0.072772,0.377820,73.0
208,SCCO,0.497796,0.076748,0.345664,74.0
94,CMI,0.154624,0.097461,0.259677,76.0
174,MAS,0.160961,0.068477,0.364583,84.0


In [ ]:
symbols = pd.read_csv("all_listed_companies.csv")
symbols = symbols['Symbol'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.15)
print(top_stocks)
world_filtered = top_stocks

Processing symbols:   0%|▏                                                           | 27/11327 [00:04<27:09,  6.93it/s]

In [ ]:
world_filtered.head(20)

## Tests

In [8]:
base_url = "https://financialmodelingprep.com/stable"
profile_url = f"{base_url}/income-statement?symbol=ROIV&apikey={api_key}"
response = requests.get(profile_url)
response.raise_for_status()
profile_data = response.json()

In [9]:
profile_data[0]

{'date': '2024-03-31',
 'symbol': 'ROIV',
 'reportedCurrency': 'USD',
 'cik': '0001635088',
 'filingDate': '2024-05-30',
 'acceptedDate': '2024-05-30 16:46:54',
 'fiscalYear': '2023',
 'period': 'FY',
 'revenue': 124795000,
 'costOfRevenue': 27964000,
 'grossProfit': 96831000,
 'researchAndDevelopmentExpenses': 501736000,
 'generalAndAdministrativeExpenses': 0,
 'sellingAndMarketingExpenses': 0,
 'sellingGeneralAndAdministrativeExpenses': 675039000,
 'otherExpenses': -5321960000,
 'operatingExpenses': -4132781000,
 'costAndExpenses': -4117221000,
 'netInterestIncome': 111647000,
 'interestIncome': 146425000,
 'interestExpense': 34778000,
 'depreciationAndAmortization': 22036000,
 'ebitda': 4310244000,
 'ebit': 4288208000,
 'nonOperatingIncomeExcludingInterest': -46192000,
 'operatingIncome': 4242016000,
 'totalOtherIncomeExpensesNet': 11414000,
 'incomeBeforeTax': 4253430000,
 'incomeTaxExpense': 22224000,
 'netIncomeFromContinuingOperations': 4231206000,
 'netIncomeFromDiscontinuedOpe

In [ ]:
top_stocks